In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time 
import pandas as pd
import json
import requests
from pyquery import PyQuery

In [2]:
def get_links(page_num):
    # Extraction of all links till june 2020
    service = Service(ChromeDriverManager().install())


    #driver = webdriver.Chrome(service=service)
    browser = webdriver.Chrome(service=service)
    urls = []
    counter = 2
    articles_link = ["Links"]
    # since the tops category has 7 pages, where link to each follows a specific pattern identified above, we can create links to them as following:
    urls.append('https://hindi.boomlive.in/fact-check')
    for i in range(2, page_num):
        urls.append( 'https://hindi.boomlive.in/fact-check/' + str(counter))
        counter += 1
    # extracting links for products in each page
    for url in urls:
        # open the url
        browser.get(url)
        # purposeful wait time to allow the website to get fully loaded
        time.sleep(4)
        # get page content
        content = browser.page_source
        soup = BeautifulSoup(content, "lxml")
        art_links = []
        # extract all the anchor i.e., <a> elements with “thumb-link” class from the page
        data_links = soup.find_all("a", {"class":"heading_link"})
        # from each <a> element, extract the URL
        for i in data_links:
            art_links.append("https://www.boomlive.in" + i['href'])

        articles_link.extend(art_links)
        # purposeful wait time to avoid sending requests in quick succession
        time.sleep(10)
        
    browser.quit()
    np.savetxt("boomlive_hindi_links_june2020.csv",
        articles_link,
        delimiter =", ",
        fmt ='% s')
    return

In [3]:
def restore_unicode(mangled):
    return mangled.encode('latin1','ignore').decode('utf8', 'replace')
def get_article_info(pq):
    headline = pq("h1.entry-title").text()
    print(headline)
    datestr = pq('span.date>span').text().split('Updated')[0]
    print(datestr)
    datestr = parse(datestr).astimezone(pytz.timezone('Asia/Calcutta')).strftime("%B %d, %Y")
    author_name = pq('a.author-name').text()
    author_link = pq('a.author-name').attr['href']
    article_info = {
        "headline": restore_unicode(headline),
        "author": restore_unicode(author_name),
        "author_link": restore_unicode(author_link),
        "date_updated": restore_unicode(datestr),
    }
    return article_info

def get_article_content(pq):

    content = {
        "text": [],
        "video": [],
        "image": [],
        "tweet": [],
        "facebook": [],
        "instagram": [],
    }

    ## text content
    content['text'] = restore_unicode(pq('div.story').text())

    ## images
    images = pq.find('figure>img')
    images += pq.find('.image-and-caption-wrapper>img')
    images += pq.find('.single-featured-thumb-container>img')
    images = list(dict.fromkeys(images))

    for i in images:
        if 'src' in i.attrib:
            content["image"].append(i.attrib["src"])

    ## video embed
    video_embed = pq.find("video>source")
    for v in video_embed:
        content["video"].append(v.attrib["src"])

    video_yt = pq.find('iframe')  # video youtube

    for v in video_yt:
        if 'lazy' in v.attrib.get('class', ''):
            continue
        content["video"].append(v.attrib["src"])

    fb = pq.find('.wp-block-embed-facebook>.fb-video')   # video fb
    for f in fb:
        content["facebook"].append(f.attrib["data-href"])
    return content

In [4]:
def get_content(dt):
    for url in dt["Links"]:
        #url = "https://www.boomlive.in/fake-news/video-of-daring-bird-rescue-in-a-chopper-is-not-from-surat-6895"
        response = requests.get(str(url))

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content of the page using BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')
            meta_tags = soup.find_all('meta')
            meta_features = {}
            for tag in meta_tags:
                if tag.get('name'):
                    meta_features[tag['name']] = tag.get('content')
            # title
            title.append(meta_features['title'])

            # Published_date
            json_data = json.loads(soup.find(type="application/ld+json").string, strict = False)
            Published_date.append(json_data['datePublished'])

            # top_image
            top_image.append(meta_features['image'])

            # image_links
            images = []
            for img in soup.findAll('img'):
                temp = img.get('src')
                if temp is not None:
                    if temp[-4:] == ".jpg":
                        images.append(temp)

            image_links.append(images)

            html_text = response.content
            pq = PyQuery(html_text)
            content = get_article_content(pq)
            Content.append(content['text'])
    # storing into a dataframe
    df = pd.DataFrame({'title': title, 'publish_date': Published_date, 'content': Content, 'top_image': top_image, 'image_links': image_links})
    df["link"] = dt["Links"]
    # saving the dataframe
    df.to_csv('boom_gujarati.csv')
    return

In [8]:
if __name__ == "__main__":
    title = []
    headline = []
    Published_date = []
    Content = []
    top_image = []
    image_links = []
    links_in_text = []
    Tags = []
    get_links(153)
    dataframe = pd.read_csv("boomlive_gujarati_links_june2020.csv")
    get_content(dataframe)